In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time


In [2]:
options=Options()
options.headless=True
options.add_argument('window-size=1920x1080')

In [3]:
website = 'https://www.topuniversities.com/university-rankings/world-university-rankings/2023'
path = Service('E:/web Scraping/chromedriver/chromedriver.exe')
driver = webdriver.Chrome(service=path,options=options)
driver.get(website)
driver.maximize_window()


In [4]:
#pagination

pagination = driver.find_element(By.XPATH,'//*[@id="alt-style-pagination"]')
page = pagination.find_elements(By.XPATH,'.//li')
last_page = int(page[2].text)
current_page = 1
    

In [5]:
University_Name=[]
University_Rank=[]
University_Score=[]
University_Location=[]
University_Links=[]
University_details=[]

In [6]:
while current_page<=last_page:
    container=driver.find_element(By.XPATH,"//*[@id='ranking-data-load']")
    box=container.find_elements(By.XPATH,".//div[@class='_qs-ranking-data-row normal-row']")
    
    for university in box:
        
        University_Rank.append(university.find_element(By.XPATH,'.//div[@class="_univ-rank hide-this-in-mobile-indi "]').text)
        University_Name.append(university.find_element(By.XPATH,".//a[@class='uni-link']").text)
        University_Score.append(university.find_element(By.XPATH,".//span[@class='overall-score-span hide-this-in-mobile-indi']").text)
        University_Location.append(university.find_element(By.XPATH,".//div[@class='location ']").text)
        University_Links.append(university.find_element(By.XPATH,".//a[@class='uni-link']").get_attribute('href'))
        
        
    try:
        next_page=driver.find_element(By.XPATH,"//a[@class='page-link next']")
        next_page.click()
    except:
        pass
    current_page=current_page+1


In [7]:
for link in University_Links:
    website = link
    path = Service('E:/web Scraping/chromedriver/chromedriver.exe')
    driver = webdriver.Chrome(service=path,options=options)
    driver.get(website)
    driver.maximize_window()
    time.sleep(3)
    info_list = []
    
    try:
        readmore = driver.find_element(By.XPATH,'//*[@id="p2-overview"]/div[3]/div/div/div[1]/p/span/a')
        readmore.click()
        time.sleep(2)
    except:
        pass
        
    information=driver.find_elements(By.XPATH,'//*[@id="p2-overview"]/div[3]/div/div/div[2]/p')
    for paragraph in information:
        info_list.append(paragraph.text)
        
    delimitor = ""
    result =  [delimitor.join(info_list)]
    University_details.extend(result)

In [8]:
df=pd.DataFrame({'University Rank':University_Rank,'University Name':University_Name,'University Score':University_Score,'Location':University_Location,'University Information':University_details})
df['University Rank'] = df['University Rank'].str.replace('=', '')

In [9]:
df.to_json('University_Details.json')

In [10]:
driver.quit()